In [1]:
import pandas as pd
import numpy as np

# ---------- STEP 1: Load Excel Sheets ----------
# Update the path as per your setup
demand_df = pd.read_csv("demand_forecasting.csv")
inventory_df = pd.read_csv("inventory_monitoring.csv")
pricing_df = pd.read_csv("pricing_optimization.csv")

In [ ]:
## standardize column names
def standardize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

demand_df = standardize_columns(demand_df)
inventory_df = standardize_columns(inventory_df)
pricing_df = standardize_columns(pricing_df)

In [ ]:
## converting into date columns
for df in [demand_df, inventory_df, pricing_df]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

In [13]:
demand_df.head(2)

product_id       date  store_id  sales_quantity  price promotions  \
0        4277 2024-01-03        48             330  24.38         No   
1        5540 2024-04-29        10             334  74.98        Yes   

  seasonality_factors    external_factors demand_trend customer_segments  \
0            Festival  Competitor Pricing   Increasing           Regular   
1             Holiday             Weather       Stable           Premium   

   sales_7d_avg  
0         330.0  
1         334.0

In [14]:
inventory_df.head(2)

product_id  store_id  stock_levels  supplier_lead_time_(days)  \
0        9286        16           700                         10   
1        2605        60            82                         11   

   stockout_frequency  reorder_point expiry_date  warehouse_capacity  \
0                  14            132  2024-01-15                1052   
1                   1            127  2024-12-16                1262   

   order_fulfillment_time_(days)  
0                              6  
1                              9

In [15]:
pricing_df.head(2)

product_id  store_id  price  competitor_prices  discounts  sales_volume  \
0        9502        13  31.61              56.14      19.68           255   
1        2068        77  35.51              63.04      16.88             5   

   customer_reviews  return_rate_(%)  storage_cost  elasticity_index  
0                 3            13.33          6.72              1.78  
1                 3             1.50          8.38              1.67

In [17]:
# 1. Demand Trends: Rolling 7-day average of sales_quantity per product_id-store_id
demand_df = demand_df.sort_values(['product_id', 'store_id', 'date'])
demand_df['sales_7d_avg'] = demand_df.groupby(['product_id', 'store_id'])['sales_quantity'] \
                                     .transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# 2. Inventory Gap: Difference between current stock and reorder point
inventory_df['inventory_gap'] = inventory_df['stock_levels'] - inventory_df['reorder_point']

# 3. Pricing Optimization: Promotion as binary flag (if not already)


In [20]:
# Promotion Flag (convert to binary if needed)
demand_df['promotions'] = demand_df['promotions'].str.strip().str.lower().map({
    'yes': 1,
    'no': 0
})

In [21]:
demand_df.to_csv("processed_demand.csv", index=False)
inventory_df.to_csv("processed_inventory.csv", index=False)
pricing_df.to_csv("processed_pricing.csv", index=False)